# Caution⚠️: to run this notebook, you need to load our finetuned model weights which stored on our google drive, if you want to test it please contact us for the model weights file.

In [ ]:
!pip install transformers

In [3]:
from google.colab import drive
drive.mount('/content/gdrive')

Mounted at /content/gdrive


In [9]:

from transformers import AutoModelForSequenceClassification,AutoModelForSeq2SeqLM
from transformers import pipeline
from transformers import AutoTokenizer
import pandas as pd
def demo_app(csv):
  t0 =[]
  t1 =[]
  t2 = []
  t3 = []
  t4 = []
  t5 = []
  pre_defined_topic = ['How to safely and correctly apply restraints to patients',
                       'How to deal with aggressive patients',
                       'Be trauma-aware during treatment', 
                       'How to deal with code white situations', 
                       'Self-care and self-protection during work',
                       'Team control and team communication during work']
  # we input the csv file
  df = pd.read_csv(csv)
  input_response = df['responses'].values
  # run through the text summarizer model
  summary_model = AutoModelForSeq2SeqLM.from_pretrained("/content/gdrive/MyDrive/Model_Summarizer")
  tokenizer = AutoTokenizer.from_pretrained("facebook/bart-large-cnn")
  summary_result = []
  for res in input_response:
    summary_pipeline  = pipeline("summarization", model = summary_model, tokenizer = tokenizer)
    # model_inputs = tokenizer(response)
    # print("********")
    # print(summary_pipeline(res)[0]['summary_text'])
    summary_result.append(summary_pipeline(res)[0]['summary_text'])


  # split the summaries into single sentences -> list
  sum_sentences = [] 
  for summary in summary_result:
    sentences = summary.split(". ")
    for s in sentences:
      sum_sentences.append(s)
  # run through the topic classifier
  topic_model = AutoModelForSequenceClassification.from_pretrained("/content/gdrive/MyDrive/Model_Classifier")
  load_tokenizer = AutoTokenizer.from_pretrained("distilbert-base-uncased")
  topic_pipeline  = pipeline("text-classification", model=topic_model, tokenizer=load_tokenizer)
  for s in sum_sentences: 
    pre_topic = topic_pipeline(s)[0]['label']
    if (pre_topic == 'LABEL_1'):
      t1.append(s)
    elif (pre_topic == 'LABEL_0'):
      t0.append(s)
    elif (pre_topic == 'LABEL_2'):
      t2.append(s)
    elif (pre_topic == 'LABEL_3'):
      t3.append(s)
    elif (pre_topic == 'LABEL_4'):
      t4.append(s)
    else:
      t5.append(s)

  # get the final result 
  total = len(t0)+ len(t1) + len(t2) + len(t3) + len(t4) + len(t5)

  # generate a new csv containing the results
  data = {pre_defined_topic[0]:" / ".join(t0),pre_defined_topic[1]:" / ".join(t1),pre_defined_topic[2]:" / ".join(t2),pre_defined_topic[3]:" / ".join(t3),pre_defined_topic[4]:" / ".join(t4),pre_defined_topic[5]:" / ".join(t5)}
  df = pd.DataFrame(data=data, index=['summary'])
  df.loc["Percentage"] = ["{0:.0f}%".format(len(t0) / total * 100), "{0:.0f}%".format(len(t1) / total * 100),"{0:.0f}%".format(len(t2) / total * 100),"{0:.0f}%".format(len(t3) / total * 100),"{0:.0f}%".format(len(t4) / total * 100),"{0:.0f}%".format(len(t5) / total * 100)]
  df.to_csv("result.csv")
  return df



In [10]:
df = demo_app('Test_intent.csv')
df

Your max_length is set to 142, but you input_length is only 130. You might consider decreasing max_length manually, e.g. summarizer('...', max_length=65)
Your max_length is set to 142, but you input_length is only 121. You might consider decreasing max_length manually, e.g. summarizer('...', max_length=60)


,How to safely and correctly apply restraints to patients,How to deal with aggressive patients,Be trauma-aware during treatment,How to deal with code white situations,Self-care and self-protection during work,Team control and team communication during work
summary,Use PINEL discontinuation strategies more effe...,Be mindful of effect of our actions has on cli...,De-escalate trauma-informed care and team comm...,More use of Red Zone and more in code white si...,Go back to appropriate self-care activities ge...,Use technique to work with team and discuss st...
Percentage,22%,25%,25%,9%,3%,16%


# Gradio Implementaion

In [ ]:
!pip install gradio

In [7]:
import gradio as gr

demo = gr.Interface(
    demo_app,
    "text",
    "dataframe"
    )

if __name__ == "__main__":
    demo.launch()

demo.launch()   

Colab notebook detected. To show errors in colab notebook, set `debug=True` in `launch()`
Note: opening Chrome Inspector may crash demo inside Colab notebooks.

To create a public link, set `share=True` in `launch()`.


<IPython.core.display.Javascript object>

Rerunning server... use `close()` to stop if you need to change `launch()` parameters.
----
Colab notebook detected. To show errors in colab notebook, set `debug=True` in `launch()`
Note: opening Chrome Inspector may crash demo inside Colab notebooks.

To create a public link, set `share=True` in `launch()`.


<IPython.core.display.Javascript object>